# Dependências ⚙

In [ ]:
!pip install pyspark==3.5.0

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import pandas as pd

sc = SparkSession.builder.master('local[*]').getOrCreate()

# Dataset 🎲

In [ ]:
df_pd = pd.read_csv("/content/salario_dataset.csv")
df_pd

,empresa,cargo,salario,nome,sobrenome,anos_de_experiencia,resumo_da_funcao
0,Cyberdyne Systems,Especialista,8931.925511,Lucas,Souza,-1.0,Apoia a equipe em atividades de rotina
1,Acme Inc.,Assistente,8400.676087,Gustavo,Oliveira,4.0,Apoia a equipe em atividades de rotina
2,Wayne Enterprises,Estagiário,2954.169316,João,Ferreira,19.0,Analisa dados e gera relatórios
3,Acme Inc.,Supervisor,4868.474440,Lucas,Santos,8.0,Auxilia a equipe em tarefas administrativas
4,Stark Enterprises,Analista,6249.850033,Fernanda,Rodrigues,6.0,Supervisiona a equipe e processos
...,...,...,...,...,...,...,...
995,Oscorp,Gerente,6775.979894,Gabriela,Pereira,6.0,Analisa dados e gera relatórios
996,Acme Inc.,Diretor,7972.585838,Gustavo,Pereira,1.0,Desenvolve soluções de software
997,Acme Inc.,Desenvolvedor,9700.553127,Lucas,Ferreira,NaN,Responsável pela gestão da equipe e projetos
998,Stark Industries,Supervisor,3594.019440,Maria,Almeida,4.0,Desenvolve soluções de software


In [ ]:
df = sc.read.csv("/content/salario_dataset.csv")
df.show()

+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|                 _c0|          _c1|               _c2|     _c3|      _c4|                _c5|                 _c6|
+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|           salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
|   Cyberdyne Systems| Especialista| 8931.925510662626|   Lucas|    Souza|               -1.0|Apoia a equipe em...|
|           Acme Inc.|   Assistente| 8400.676087373886| Gustavo| Oliveira|                4.0|Apoia a equipe em...|
|   Wayne Enterprises|   Estagiário|2954.1693163957525|    João| Ferreira|               19.0|Analisa dados e g...|
|           Acme Inc.|   Supervisor| 4868.474439934829|   Lucas|   Santos|                8.0|Auxilia a equipe ...|
|   Stark Enterprises|     Analista| 6249.850033405442|Fernanda|Rodrigue

# Análise Exploratória 🔍

Utilizando Pandas aqui pois a biblioteca tem mais funções para analisar o dataset de forma detalhada

In [ ]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   empresa              1000 non-null   object 
 1   cargo                1000 non-null   object 
 2   salario              951 non-null    float64
 3   nome                 991 non-null    object 
 4   sobrenome            990 non-null    object 
 5   anos_de_experiencia  971 non-null    float64
 6   resumo_da_funcao     1000 non-null   object 
dtypes: float64(2), object(5)
memory usage: 54.8+ KB


In [ ]:
df_pd.describe()

,salario,anos_de_experiencia
count,951.000000,971.000000
mean,5926.859377,9.853759
std,2423.008110,5.589869
min,-1.000000,-1.000000
25%,3909.619853,5.000000
50%,5959.769344,10.000000
75%,8038.717512,15.000000
max,9969.040686,19.000000


In [ ]:
df_pd.count()


,0
empresa,1000
cargo,1000
salario,951
nome,991
sobrenome,990
anos_de_experiencia,971
resumo_da_funcao,1000


In [ ]:
df_pd['salario'].quantile(q=0.91)

9233.176941100975

In [ ]:
(df_pd['nome'] + ' ' + df_pd['sobrenome']).value_counts()

,count
Isabela Rodrigues,19
Gabriela Pereira,19
Ana Santos,18
Ana Souza,16
Maria Rodrigues,16
...,...
Matheus Martins,5
João Pereira,4
Maria Silva,4
Pedro Pereira,4


In [ ]:
columns = ['empresa', 'cargo', 'nome', 'sobrenome', 'resumo_da_funcao']
for i in columns:
  print(i, df_pd[i].unique(), '\n')

empresa ['Cyberdyne Systems' 'Acme Inc.' 'Wayne Enterprises' 'Stark Enterprises'
 'Weyland-Yutani' 'Globex Corp.' 'Stark Industries' 'Umbrella Corporation'
 'Oscorp' 'Aperture Science' 'lnnNSf5YDt'] 

cargo ['Especialista' 'Assistente' 'Estagiário' 'Supervisor' 'Analista'
 'Diretor' 'Desenvolvedor' 'Gerente' 'Coordenador' 'Consultor'
 's2iOR6wzbFZtK5T'] 

nome ['Lucas' 'Gustavo' 'João' 'Fernanda' 'Gabriela' 'Pedro' 'Maria' 'Ana'
 'Isabela' nan 'Matheus'] 

sobrenome ['Souza' 'Oliveira' 'Ferreira' 'Santos' 'Rodrigues' 'Silva' nan 'Pereira'
 'Costa' 'Martins' 'Almeida'] 

resumo_da_funcao ['Apoia a equipe em atividades de rotina'
 'Analisa dados e gera relatórios'
 'Auxilia a equipe em tarefas administrativas'
 'Supervisiona a equipe e processos' 'Desenvolve soluções de software'
 'Lidera a estratégia e tomada de decisões'
 'Responsável pela gestão da equipe e projetos'
 'Coordena atividades e projetos'
 'Especialista em estratégia e planejamento'
 'Presta consultoria em áreas específica

# Limpeza dos dados 🧹

## Renomeia colunas



In [ ]:
df = (

    df
    .withColumnRenamed("_c0", "empresa")
    .withColumnRenamed("_c1", "cargo")
    .withColumnRenamed("_c2", "salario")
    .withColumnRenamed("_c3", "nome")
    .withColumnRenamed("_c4", "sobrenome")
    .withColumnRenamed("_c5", "anos_de_experiencia")
    .withColumnRenamed("_c6", "resumo_da_funcao")



)

## Exclui primeira linha

In [ ]:
df = (

      df
      .withColumn("index", F.monotonically_increasing_id())
      .filter(F.col("index") != 0)
      .drop("index")

)

## Retira as linhas com valores nulos

In [ ]:
df = (
    df
    .na.drop()

)

## Retira as duplicatas

In [ ]:
df = df.dropDuplicates()
df.count()

908

## Define o maior tempo de experiência como linha correta

In [ ]:
df = (

    df
    .withColumn("nome_completo", F.concat_ws(" ", F.col("nome"), F.col("sobrenome")))


)


window_spec = Window.partitionBy("nome_completo").orderBy(F.col("anos_de_experiencia").desc())
df = df.withColumn("rank", F.row_number().over(window_spec))

df = df.filter(F.col("rank") == 1).drop("rank", "nome_completo")

df.show()

+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|           salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|              Oscorp| Especialista| 3277.746073162348|     Ana|  Almeida|                8.0|Responsável pela ...|
|    Stark Industries|   Supervisor| 6764.052733928409|     Ana|    Costa|                7.0|Supervisiona a eq...|
|Umbrella Corporation|   Assistente|3231.0381390691546|     Ana| Ferreira|                5.0|Apoia a equipe em...|
|    Aperture Science|      Diretor| 7562.487969540558|     Ana|  Martins|                9.0|Apoia a equipe em...|
|    Aperture Science|      Diretor| 9921.601223220283|     Ana| Oliveira|                9.0|Lidera a estratég...|
|   Wayne Enterprises|      Gerente| 8122.239940784619|     Ana|  Pereir

## Retira Strings indesejadas

In [ ]:
df = (

      df
      .filter(F.col("empresa") != 'lnnNSf5YDt')
      .filter(F.col("cargo") != 's2iOR6wzbFZtK5T')

)

df.show()

+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|           salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|              Oscorp| Especialista| 3277.746073162348|     Ana|  Almeida|                8.0|Responsável pela ...|
|    Stark Industries|   Supervisor| 6764.052733928409|     Ana|    Costa|                7.0|Supervisiona a eq...|
|Umbrella Corporation|   Assistente|3231.0381390691546|     Ana| Ferreira|                5.0|Apoia a equipe em...|
|    Aperture Science|      Diretor| 7562.487969540558|     Ana|  Martins|                9.0|Apoia a equipe em...|
|    Aperture Science|      Diretor| 9921.601223220283|     Ana| Oliveira|                9.0|Lidera a estratég...|
|   Wayne Enterprises|      Gerente| 8122.239940784619|     Ana|  Pereir

## Define o tempo de experiência para INT positivo

In [ ]:
df = (

      df
      .filter(F.col("anos_de_experiencia") > 0)
      .withColumn("anos_de_experiencia", F.col("anos_de_experiencia").cast(IntegerType()))

)

df.show()

+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|           salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
+--------------------+-------------+------------------+--------+---------+-------------------+--------------------+
|              Oscorp| Especialista| 3277.746073162348|     Ana|  Almeida|                  8|Responsável pela ...|
|    Stark Industries|   Supervisor| 6764.052733928409|     Ana|    Costa|                  7|Supervisiona a eq...|
|Umbrella Corporation|   Assistente|3231.0381390691546|     Ana| Ferreira|                  5|Apoia a equipe em...|
|    Aperture Science|      Diretor| 7562.487969540558|     Ana|  Martins|                  9|Apoia a equipe em...|
|    Aperture Science|      Diretor| 9921.601223220283|     Ana| Oliveira|                  9|Lidera a estratég...|
|   Wayne Enterprises|      Gerente| 8122.239940784619|     Ana|  Pereir

## Define o salario para apenas duas casas decimais

In [ ]:
df = (

      df
      .withColumn("salario", F.round("salario", 2))
)

df.show()

+--------------------+-------------+-------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
+--------------------+-------------+-------+--------+---------+-------------------+--------------------+
|              Oscorp| Especialista|3277.75|     Ana|  Almeida|                  8|Responsável pela ...|
|    Stark Industries|   Supervisor|6764.05|     Ana|    Costa|                  7|Supervisiona a eq...|
|Umbrella Corporation|   Assistente|3231.04|     Ana| Ferreira|                  5|Apoia a equipe em...|
|    Aperture Science|      Diretor|7562.49|     Ana|  Martins|                  9|Apoia a equipe em...|
|    Aperture Science|      Diretor| 9921.6|     Ana| Oliveira|                  9|Lidera a estratég...|
|   Wayne Enterprises|      Gerente|8122.24|     Ana|  Pereira|                  6|Auxilia a equipe ...|
|           Acme Inc.|Desenvolvedor|2098.69|     Ana|Ro

## Remove os outliers de salario usando IQR

In [ ]:
quantiles = df.approxQuantile("salario", [0.25, 0.75], 0.05)
q1, q3 = quantiles[0], quantiles[1]
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df = df.filter(~(F.col("salario") < lower_bound) | (F.col("salario") > upper_bound))

df.show()

+--------------------+-------------+-------+--------+---------+-------------------+--------------------+
|             empresa|        cargo|salario|    nome|sobrenome|anos_de_experiencia|    resumo_da_funcao|
+--------------------+-------------+-------+--------+---------+-------------------+--------------------+
|              Oscorp| Especialista|3277.75|     Ana|  Almeida|                  8|Responsável pela ...|
|    Stark Industries|   Supervisor|6764.05|     Ana|    Costa|                  7|Supervisiona a eq...|
|Umbrella Corporation|   Assistente|3231.04|     Ana| Ferreira|                  5|Apoia a equipe em...|
|    Aperture Science|      Diretor|7562.49|     Ana|  Martins|                  9|Apoia a equipe em...|
|    Aperture Science|      Diretor| 9921.6|     Ana| Oliveira|                  9|Lidera a estratég...|
|   Wayne Enterprises|      Gerente|8122.24|     Ana|  Pereira|                  6|Auxilia a equipe ...|
|           Acme Inc.|Desenvolvedor|2098.69|     Ana|Ro